Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

## Imports

In [ ]:
from random import choices, choice, random, randrange
from dataclasses import dataclass
from copy import copy
from tqdm.auto import tqdm

import lab9_lib

## EA Strategy

### Hyper-parameters

In [ ]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 100000
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .1
GENOTYPE_SIZE = 1000
PROBLEM_SIZE = 1
N_GENERATIONS = 10

### Initializations

In [ ]:
fitness = lab9_lib.make_problem(PROBLEM_SIZE)

### Classes and methods

In [ ]:
@dataclass
class Individual:
    genotype: list[int]
    fitness: float


def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion


def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randrange(0, GENOTYPE_SIZE)

    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None

    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randrange(0, GENOTYPE_SIZE)
    offspring = Individual(
        fitness=None, genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:]
    )
    assert len(offspring.genotype) == GENOTYPE_SIZE
    return offspring


def two_cuts_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point1 = randrange(0, GENOTYPE_SIZE)
    cut_point2 = randrange(0, GENOTYPE_SIZE)

    if cut_point1 < cut_point2:
        offspring = choice(
            [
                Individual(
                    fitness=None,
                    genotype=ind1.genotype[:cut_point1]
                    + ind2.genotype[cut_point1:cut_point2]
                    + ind1.genotype[cut_point2:],
                ),
                Individual(
                    fitness=None,
                    genotype=ind1.genotype[:cut_point1]
                    + ind2.genotype[cut_point1:cut_point2]
                    + ind2.genotype[cut_point2:],
                ),
            ]
        )
    elif cut_point1 > cut_point2:
        offspring = choice(
            [
                Individual(
                    fitness=None,
                    genotype=ind1.genotype[:cut_point2]
                    + ind2.genotype[cut_point2:cut_point1]
                    + ind1.genotype[cut_point1:],
                ),
                Individual(
                    fitness=None,
                    genotype=ind1.genotype[:cut_point2]
                    + ind2.genotype[cut_point2:cut_point1]
                    + ind2.genotype[cut_point1:],
                ),
            ]
        )
    else:
        offspring = Individual(
            fitness=None,
            genotype=ind1.genotype[:cut_point1] + ind2.genotype[cut_point1:],
        )
    assert len(offspring.genotype) == GENOTYPE_SIZE
    return offspring

### Population generation

In [ ]:
population = [
    Individual(genotype=choices([0, 1], k=GENOTYPE_SIZE), fitness=None)
    for _ in range(POPULATION_SIZE)
]

for ind in population:
    ind.fitness = fitness(ind.genotype)

population

### Offsprings generation

In [ ]:
with tqdm(total=N_GENERATIONS * OFFSPRING_SIZE) as pbar:
    
    for generation in range(N_GENERATIONS):
        offspring = list()

        for counter in range(OFFSPRING_SIZE):
            if random() < MUTATION_PROBABILITY:
                p = select_parent(population)
                o = mutate(p)
            else:
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = one_cut_xover(p1, p2)

            offspring.append(o)
        
            pbar.update(1)
        
        for i in offspring:
            i.fitness = fitness(i.genotype)
        
        population.extend(offspring)
        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
        print(population[0].fitness)
        
        if population[0].fitness == 1.0:
            break
    
print(fitness.calls)

In [ ]:
population

## Random strategy

In [ ]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")
print(fitness.calls)